**Importing needed Libraries and Packages for Training**

In [1]:
import os
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

#XGBoost
try:
    from xgboost import XGBRegressor
except Exception as e:
    raise ImportError("Install xgboost: pip install xgboost") from e

#SHAP
try:
    import shap
except Exception:
    shap = None
    print("shap not installed. Install with: pip install shap to get SHAP plots")

RANDOM_STATE = 42
CLEAN_CSV = "/content/drive/MyDrive/Crop_Prediction_Project/Cleaned_dataset.csv"  # path to cleaned dataset
OUTPUT_DIR = "/content/drive/MyDrive/Crop_Prediction_Project"; os.makedirs(OUTPUT_DIR, exist_ok=True)


### 1. Load data

This step loads the cleaned dataset from the specified CSV file path into a pandas DataFrame.

In [2]:
df = pd.read_csv(CLEAN_CSV)
print("Loaded rows:", len(df))
print(df.columns.tolist())

Loaded rows: 48733
['Crop', 'Season', 'Soil_Type', 'N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall', 'Yield_t_ha']


**Observation:** The dataset has been loaded successfully, and the number of rows and column names are printed.

### 2. Prepare features & target

In this step, the features (independent variables) and the target variable (dependent variable) are separated. Categorical and numerical columns are identified, and the data is split into training and testing sets. The splitting is stratified by the 'Crop' column to maintain the distribution of crops in both sets.

In [3]:
# --------- 2. Prepare features & target ----------
target_col = "Yield_t_ha"
cat_cols = ["Crop", "Season", "Soil_Type"]
num_cols = [c for c in df.columns if c not in cat_cols + [target_col]]

X = df[cat_cols + num_cols].copy()
y = df[target_col].copy()

# Train-test split (stratify by Crop to keep crop distribution similar)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=RANDOM_STATE, stratify=X['Crop']
)

**Observation:** The features (X) and target (y) are prepared, and the data is split into training and testing sets. The shapes of the resulting arrays are not explicitly printed, but the process is completed.

### 3. Preprocessing pipelines

This step defines two preprocessing pipelines: one for tree-based models (like Random Forest and XGBoost) and one for linear models (like Linear Regression). The tree-based pipeline uses One-Hot Encoding for categorical features and passes numerical features through. The linear pipeline uses One-Hot Encoding and StandardScaler for numerical features.

In [5]:
# --------- 3. Preprocessing pipelines ----------
# For tree models we'll one-hot encode categorical variables and pass numeric as-is
tree_preprocessor = ColumnTransformer(
    transformers=[
        ("ohe", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols),
    ],
    remainder="drop",
)

# For linear regression we scale numeric features and OHE the categoricals
linear_preprocessor = ColumnTransformer(
    transformers=[
        ("ohe", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("scale", StandardScaler(), num_cols),
    ],
    remainder="drop",
)

**Observation:** Two `ColumnTransformer` objects, `tree_preprocessor` and `linear_preprocessor`, are created and configured for different model types. These are ready to be used in the model pipelines.

### 4. Define models & pipelines

This step defines the machine learning models to be used and creates pipelines that combine the preprocessing steps with the models. Pipelines are created for Linear Regression, Random Forest, and XGBoost.

In [6]:
# --------- 4. Define models & pipelines ----------
models = {}

# Linear Regression pipeline
lin_pipe = Pipeline([("pre", linear_preprocessor), ("lin", LinearRegression())])
models["LinearRegression"] = lin_pipe

# Random Forest pipeline
rf_pipe = Pipeline([("pre", tree_preprocessor), ("rf", RandomForestRegressor(
    n_estimators=300, n_jobs=-1, random_state=RANDOM_STATE
))])
models["RandomForest"] = rf_pipe

# XGBoost pipeline
xgb_pipe = Pipeline([("pre", tree_preprocessor), ("xgb", XGBRegressor(
    n_estimators=1000, learning_rate=0.05, max_depth=8,
    objective="reg:squarederror", random_state=RANDOM_STATE, n_jobs=-1
))])
models["XGBoost"] = xgb_pipe

# Decision Tree Regressor
from sklearn.tree import DecisionTreeRegressor
dt_pipe = Pipeline([("pre", tree_preprocessor), ("dt", DecisionTreeRegressor(random_state=RANDOM_STATE))])
models["DecisionTree"] = dt_pipe

# Support Vector Regressor (Linear)
from sklearn.svm import LinearSVR
# Need to scale numerical features for SVR
svr_pipe = Pipeline([("pre", linear_preprocessor), ("svr", LinearSVR(random_state=RANDOM_STATE, max_iter=10000))])
models["LinearSVR"] = svr_pipe

# Gradient Boosting Regressor
from sklearn.ensemble import GradientBoostingRegressor
gbr_pipe = Pipeline([("pre", tree_preprocessor), ("gbr", GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=RANDOM_STATE))])
models["GradientBoosting"] = gbr_pipe

**Observation:** Three pipelines (`lin_pipe`, `rf_pipe`, and `xgb_pipe`) are defined, each incorporating the appropriate preprocessor and model. These pipelines are stored in the `models` dictionary.

### 5. Train & evaluate

This step iterates through the defined models, trains each pipeline on the training data (`X_train`, `y_train`), and evaluates their performance on the test data (`X_test`, `y_test`). Evaluation metrics calculated include R-squared (R2), Mean Absolute Error (MAE), Mean Squared Error (MSE), and Root Mean Squared Error (RMSE). The results are stored in a DataFrame and saved to a CSV file.

In [7]:
# --------- 5. Train & evaluate ----------
results = []
fitted_models = {}

for name, pipe in models.items():
    print(f"\nTraining {name} ...")
    pipe.fit(X_train, y_train)
    pred = pipe.predict(X_test)
    r2 = r2_score(y_test, pred)
    mae = mean_absolute_error(y_test, pred)
    mse = mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    results.append({"model": name, "r2": r2, "mae": mae, "rmse": rmse})
    fitted_models[name] = pipe
    print(f"{name} => R2: {r2:.4f} | MAE: {mae:.4f} | RMSE: {rmse:.4f}")

# Save metrics
metrics_df = pd.DataFrame(results).sort_values("r2", ascending=False)
metrics_df.to_csv(os.path.join(OUTPUT_DIR, "model_metrics.csv"), index=False)
print("\nSaved metrics to:", os.path.join(OUTPUT_DIR, "model_metrics.csv"))
print(metrics_df)


Training LinearRegression ...
LinearRegression => R2: 0.8153 | MAE: 1.4453 | RMSE: 2.2337

Training RandomForest ...
RandomForest => R2: 0.8018 | MAE: 1.4884 | RMSE: 2.3136

Training XGBoost ...
XGBoost => R2: 0.7858 | MAE: 1.4886 | RMSE: 2.4055

Training DecisionTree ...
DecisionTree => R2: 0.7659 | MAE: 1.5124 | RMSE: 2.5145

Training LinearSVR ...
LinearSVR => R2: 0.6938 | MAE: 1.4833 | RMSE: 2.8759

Training GradientBoosting ...
GradientBoosting => R2: 0.8029 | MAE: 1.4657 | RMSE: 2.3075

Saved metrics to: /content/drive/MyDrive/Crop_Prediction_Project/model_metrics.csv
              model        r2       mae      rmse
0  LinearRegression  0.815274  1.445271  2.233668
5  GradientBoosting  0.802856  1.465697  2.307526
1      RandomForest  0.801818  1.488359  2.313589
2           XGBoost  0.785767  1.488581  2.405458
3      DecisionTree  0.765906  1.512419  2.514488
4         LinearSVR  0.693786  1.483336  2.875856


**Observation:** Each model is trained and evaluated. The R2, MAE, and RMSE scores for all the models are printed to the console and saved to `model_metrics.csv`. The output shows the performance of Linear Regression, Random Forest, XGBoost, Decision Tree, Linear SVR, and Gradient Boosting, with Linear Regression currently being the best-performing model based on R2.

### 6. Interpret XGBoost model (SHAP)

This step uses the SHAP (SHapley Additive exPlanations) library to interpret the XGBoost model. It samples a subset of the test data, transforms it using the tree preprocessor, and calculates SHAP values. A summary plot and dependence plots for the top features are generated and saved as PNG files. The code includes a fallback mechanism if the preferred `TreeExplainer` fails.

In [8]:
# Robust SHAP block that handles sparse/dense transform outputs
import os, numpy as np, matplotlib.pyplot as plt, scipy.sparse as sp
import shap

OUT = OUTPUT_DIR  # reuse your OUTPUT_DIR
os.makedirs(OUT, exist_ok=True)

# 1) Sample for SHAP (smaller to be safe)
sample_n = min(500, X_test.shape[0])   # 500 is safer on limited RAM; increase if you have memory
X_test_sample = X_test.sample(n=sample_n, random_state=42)

# Preprocessor & model from your fitted pipeline
pre = fitted_models["XGBoost"].named_steps["pre"]
model_xgb = fitted_models["XGBoost"].named_steps["xgb"]

# 2) Transform test sample using preprocessor
X_test_trans = pre.transform(X_test_sample)

# If transform returned a scipy sparse matrix, convert to dense safely
if sp.issparse(X_test_trans):
    print("Detected sparse matrix; converting to dense with toarray() (may use memory).")
    X_test_trans = X_test_trans.toarray()
else:
    # If it's an object array or nested arrays, try converting safely
    X_test_trans = np.asarray(X_test_trans)

# Ensure numeric dtype
X_test_trans = X_test_trans.astype(np.float32)

# 3) Build feature names matching transformed columns
cat_cols = ["Crop", "Season", "Soil_Type"]
num_cols = [c for c in X.columns if c not in cat_cols]

try:
    ohe = pre.named_transformers_["ohe"]
    # sklearn >=1.0
    ohe_names = list(ohe.get_feature_names_out(cat_cols))
except Exception:
    try:
        ohe_names = list(ohe.get_feature_names(cat_cols))
    except Exception:
        # Last resort: combine cat name + unique values (may not match ordering exactly)
        ohe_names = []
        for c in cat_cols:
            vals = list(X[c].astype(str).unique())
            vals = [str(v).replace(" ", "_") for v in vals]
            ohe_names += [f"{c}_{v}" for v in vals]

feature_names = ohe_names + num_cols

# Sanity: check shapes
if X_test_trans.shape[1] != len(feature_names):
    print("WARNING: feature name count mismatch ->", X_test_trans.shape[1], "vs", len(feature_names))
    # Try to proceed but SHAP may still complain. You can inspect feature_names or regenerate them.

# 4) Try TreeExplainer with Booster (preferred).
saved = False
try:
    booster = model_xgb.get_booster()
    explainer = shap.TreeExplainer(booster)
    shap_vals = explainer.shap_values(X_test_trans)
    plt.figure(figsize=(10,6))
    shap.summary_plot(shap_vals, X_test_trans, feature_names=feature_names, show=False)
    plt.tight_layout()
    outp = os.path.join(OUT, "xgb_shap_summary_booster.png")
    plt.savefig(outp); plt.close()
    print("Saved SHAP summary (booster) to", outp)
    saved = True
except Exception as e:
    print("TreeExplainer(booster) failed:", repr(e))

# 5) Fallback: shap.Explainer using a predict wrapper (robust)
if not saved:
    print("Falling back to shap.Explainer using model.predict (robust fallback).")
    # Build small transformed train sample for masker
    train_sample_n = min(300, X_train.shape[0])
    X_train_sample = X_train.sample(n=train_sample_n, random_state=42)
    X_train_trans = pre.transform(X_train_sample)
    if sp.issparse(X_train_trans):
        X_train_trans = X_train_trans.toarray()
    X_train_trans = np.asarray(X_train_trans).astype(np.float32)

    # Create predict function that accepts transformed arrays
    def xgb_predict_transformed(mat):
        mat = np.asarray(mat).astype(np.float32)
        return model_xgb.predict(mat)

    # Create an Independent masker on the transformed train data
    try:
        masker = shap.maskers.Independent(X_train_trans)
    except Exception:
        masker = shap.maskers.Independent(X_train_trans.astype(np.float32))

    # Initialize explainer with the predict callable
    explainer = shap.Explainer(xgb_predict_transformed, masker=masker, feature_names=feature_names)
    shap_res = explainer(X_test_trans)  # returns Explanation object
    # Save summary plot
    plt.figure(figsize=(10,6))
    shap.summary_plot(shap_res.values, X_test_trans, feature_names=feature_names, show=False)
    plt.tight_layout()
    outp = os.path.join(OUT, "xgb_shap_summary_explainer.png")
    plt.savefig(outp); plt.close()
    print("Saved SHAP summary (Explainer) to", outp)

# 6) Dependence plots for top features (if SHAP succeeded)
try:
    if 'shap_vals' in locals():
        S = np.abs(shap_vals).mean(axis=0)
    else:
        S = np.abs(shap_res.values).mean(axis=0)
    top_idx = np.argsort(S)[-4:][::-1]
    for i in top_idx:
        fname = feature_names[i] if i < len(feature_names) else f"f{i}"
        plt.figure(figsize=(6,4))
        try:
            shap.dependence_plot(i, shap_vals if 'shap_vals' in locals() else shap_res.values,
                                 X_test_trans, feature_names=feature_names, show=False)
        except Exception:
            # fallback to name-based dependence
            shap.dependence_plot(fname, shap_vals if 'shap_vals' in locals() else shap_res.values,
                                 X_test_trans, feature_names=feature_names, show=False)
        plt.tight_layout()
        outp = os.path.join(OUT, f"xgb_shap_dependence_{fname.replace(' ', '_')}.png")
        plt.savefig(outp); plt.close()
        print("Saved dependence for", fname, "->", outp)
except Exception as e_dep:
    print("Could not create dependence plots:", repr(e_dep))

print("Done. Check saved files in:", OUT)


Detected sparse matrix; converting to dense with toarray() (may use memory).
TreeExplainer(booster) failed: ValueError("could not convert string to float: '[5.3128877E0]'")
Falling back to shap.Explainer using model.predict (robust fallback).


PermutationExplainer explainer: 501it [05:01,  1.63it/s]
/tmp/ipython-input-279426551.py:99: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(shap_res.values, X_test_trans, feature_names=feature_names, show=False)


Saved SHAP summary (Explainer) to /content/drive/MyDrive/Crop_Prediction_Project/xgb_shap_summary_explainer.png
Saved dependence for temperature -> /content/drive/MyDrive/Crop_Prediction_Project/xgb_shap_dependence_temperature.png
Saved dependence for N -> /content/drive/MyDrive/Crop_Prediction_Project/xgb_shap_dependence_N.png
Saved dependence for ph -> /content/drive/MyDrive/Crop_Prediction_Project/xgb_shap_dependence_ph.png
Saved dependence for P -> /content/drive/MyDrive/Crop_Prediction_Project/xgb_shap_dependence_P.png
Done. Check saved files in: /content/drive/MyDrive/Crop_Prediction_Project


<Figure size 600x400 with 0 Axes>

<Figure size 600x400 with 0 Axes>

<Figure size 600x400 with 0 Axes>

<Figure size 600x400 with 0 Axes>

**Observation:** The SHAP analysis for the XGBoost model is performed. The code successfully falls back to using `shap.Explainer` with a predict wrapper. A SHAP summary plot and dependence plots for the top 4 features (temperature, N, ph, and P) are saved as PNG files.

### 7. Interpret Linear Regression (coefficients)

This step interprets the Linear Regression model by examining the coefficients assigned to each feature. It retrieves the feature names from the linear preprocessor and extracts the coefficients from the trained linear model. The coefficients are stored in a DataFrame, sorted by their absolute values in descending order, and saved to a CSV file.

In [11]:
lin = fitted_models["LinearRegression"]
# get feature names for the linear preprocessor
pre_lin = lin.named_steps["pre"]
try:
    lin_ohe_names = pre_lin.named_transformers_["ohe"].get_feature_names_out(cat_cols)
except Exception:
    lin_ohe_names = pre_lin.named_transformers_["ohe"].get_feature_names(cat_cols)
lin_feature_names = list(lin_ohe_names) + num_cols

coeffs = lin.named_steps["lin"].coef_
coef_df = pd.DataFrame({"feature": lin_feature_names, "coef": coeffs})
coef_df = coef_df.reindex(coef_df.coef.abs().sort_values(ascending=False).index)
coef_df.to_csv(os.path.join(OUTPUT_DIR, "linear_coefficients.csv"), index=False)
print("Saved linear coefficients to", os.path.join(OUTPUT_DIR, "linear_coefficients.csv"))



Saved linear coefficients to /content/drive/MyDrive/Crop_Prediction_Project/linear_coefficients.csv


**Observation:** The coefficients for the Linear Regression model are calculated and saved to `linear_coefficients.csv`. The output indicates that the file has been successfully saved.

### 8. Select best-performing model and save

This step selects the best-performing model based on the R2 score from the evaluation results. The selected model pipeline is then saved to a joblib file. A summary of the model selection process, including the metrics and the path to the saved model, is also saved to a text file.

In [12]:
# Selection criteria: highest R2 (you can change to lowest RMSE if you prefer)
best_row = metrics_df.iloc[0]
best_model_name = best_row['model']
best_model_pipe = fitted_models[best_model_name]
best_model_path = os.path.join(OUTPUT_DIR, f"best_model_{best_model_name}.joblib")
joblib.dump(best_model_pipe, best_model_path)
print(f"Best model: {best_model_name} saved to {best_model_path}")

# Save a human readable summary
with open(os.path.join(OUTPUT_DIR, "selection_summary.txt"), "w") as f:
    f.write("Model comparison (sorted by R2):\n")
    f.write(metrics_df.to_string(index=False))
    f.write("\n\nSelected model: " + best_model_name + "\n")
    f.write("Saved model path: " + best_model_path + "\n")
print("Saved selection summary to:", os.path.join(OUTPUT_DIR, "selection_summary.txt"))

Best model: LinearRegression saved to /content/drive/MyDrive/Crop_Prediction_Project/best_model_LinearRegression.joblib
Saved selection summary to: /content/drive/MyDrive/Crop_Prediction_Project/selection_summary.txt


**Observation:** The Linear Regression model is identified as the best-performing model based on the R2 score. The trained Linear Regression pipeline is saved as `best_model_LinearRegression.joblib`, and a selection summary is saved to `selection_summary.txt`.

### 9. Test the best model with a sample input

This step demonstrates how to use the selected best model (Linear Regression) to predict the yield for a sample input.

In [14]:
# --------- 9. Test the best model with a sample input ----------

# Define a sample input based on the dataset's features
# Replace with your desired values for prediction
# sample_input = {
#     'Crop': 'rice',
#     'Season': 'Summer',
#     'Soil_Type': 'loamy',
#     'N': 80.0,
#     'P': 50.0,
#     'K': 30.0,
#     'temperature': 25.0,
#     'humidity': 80.0,
#     'ph': 6.0,
#     'rainfall': 200.0
# }

# Convert the sample input to a DataFrame
# sample_df = pd.DataFrame([sample_input])

# Get the best fitted model
best_model_pipe = fitted_models[best_model_name]

# Select a sample from the test set
sample_index = 0 # You can change this index to pick a different sample
sample_input_from_test = X_test.iloc[[sample_index]]
actual_yield = y_test.iloc[sample_index]


# Make a prediction using the best model
predicted_yield = best_model_pipe.predict(sample_input_from_test)

print(f"Sample Input (from test set):")
print(sample_input_from_test)
print(f"\nActual Yield: {actual_yield:.4f} t/ha")
print(f"Predicted Yield: {predicted_yield[0]:.4f} t/ha")

# To calculate accuracy, you would need an actual yield value for this sample input.
# You can then compare the predicted_yield to the actual yield.
# Example:
# accuracy = 1 - abs(predicted_yield[0] - actual_yield) / actual_yield
# print(f"Accuracy: {accuracy:.4f}")

Sample Input (from test set):
          Crop Season Soil_Type     N     P     K  temperature   humidity  \
9902  chickpea   Rabi     loamy  32.0  60.0  83.0    19.691417  19.442254   

            ph   rainfall  
9902  8.829273  91.760716  

Actual Yield: 5.1667 t/ha
Predicted Yield: 4.6975 t/ha


**Observation:** A sample input from the test set was used to test the best model (Linear Regression). The actual yield for this sample was 5.1667 t/ha, and the model predicted a yield of 4.6975 t/ha. This provides a direct comparison between the model's prediction and the true value for a specific data point.

To further improve the best model (Linear Regression), you can consider the following:

*   **Feature Engineering:** Create new features from existing ones (e.g., interactions, polynomial terms).
*   **Handling Categorical Features:** Explore alternative encoding methods beyond one-hot encoding.
*   **Regularization:** Apply L1 (Lasso) or L2 (Ridge) regularization to prevent overfitting.
*   **Polynomial Features:** Introduce polynomial features to capture non-linear relationships.
*   **Outlier Detection and Handling:** Identify and address outliers in the dataset.
*   **Hyperparameter Tuning:** Tune the parameters of the preprocessing steps.
*   **Alternative Linear Models:** Experiment with other linear models like Ridge, Lasso, or Elastic Net.
*   **Ensemble Methods:** Combine predictions from multiple models.